# ソルバーの実行

## 準備

In [1]:
import gurobi as gp
import datetime
import random
import numpy as np
random.seed(1)
import os

In [2]:
def group(i,l): # アイテムiがグループGに属しているかどうかをバイナリで返す関数
    for j in l:
        for k in j:
            if k==i:
                return 1
            else:
                return 0

In [3]:
def group(i,l): # アイテムiがグループGに属しているかどうかをバイナリで返す関数
    if i in l:
        return 1
    else:
        return 0

In [4]:
tl = 1000

In [5]:
def solve(moel_name,n,w,C,E,M,r,bin_cost,tl):
    model = gp.Model(name = model_name)
    model.setParam('TimeLimit', tl) #制限時間
    #解
    ans_x = {}
    ans_y = {}
    ans_b = {}
    #定数定義
    m = len(M)
    z = {}
    #変数定義
    x = {}
    y = {}
    b = {}
    for l in range(m):
        for k in range(n):
            y[k] = model.addVar(lb=0, ub=1, vtype='B', name='y[%s]'%k)
            b[k,l] = model.addVar(lb=0, ub=1, vtype='B', name='b[%s,%s]'%(k,l))
            for i in range(n):
                x[i,k] = model.addVar(lb=0, ub=1, vtype='B', name='x[%s,%s]'%(i,k))
                z[i,l] = group(i,M[l])
                #if l==0 and z[i,l]==1:
                    #print(i)
                
    #print(z)
    model.update()
        
    #目的関数
    ##model.setObjective(gp.quicksum(bin_cost*y[k] for k in range(n)) + gp.quicksum(r[l]*gp.quicksum(b[k,l] for k in range(n)) for l in range(m)) - sum(r), sense=gp.GRB.MINIMIZE)
    model.setObjective(gp.quicksum(y[k] for k in range(n)) + gp.quicksum(gp.quicksum(r[l]*b[k,l] for l in range(m)) for k in range(n)) - sum(r), sense=gp.GRB.MINIMIZE)

    #制約式
    for k in range(n):
        model.addConstr(gp.quicksum(w[i]*x[i,k] for i in range(n))<=C*y[k], 'Binlimit[%s]'%k) #ビンの容量を超えない
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,k] for k in range(n))==1, 'Assign[%s]'%i) #すべてのアイテムはビンに詰め込まれている
        for k in range(n):
            model.addConstr(x[i,k]<=y[k], 'Strong[%s,%s]'%(i,k)) #強い定式化
    for e in E:
        for k in range(n):
            model.addConstr(x[e[0],k]+x[e[1],k]<=1,'Conflict[%s,%s]'%(e[0],e[1])) #コンフリクト
    for k in range(n):
        for l in range(m):
            for j in range(n):
                model.addConstr(b[k,l]>=x[j,k]*z[j,l],'Group[%s]'%l) #同じグループに属するアイテムは同じビンに割り当てられる
    
    #解の出力
    ##print("↓点線の間に、Gurobi Optimizerからログが出力")
    ##print("-" * 100)

    model.optimize()
    
    if model.Status == gp.GRB.INFEASIBLE:
        pass
    else:
        model.write(f"{model_name}.sol")
    
    ##print("-" * 100)
    ##print()
    
    """if model.Status == gp.GRB.OPTIMAL:
        for v in model.getVars():
            if "x" in v.varName:
                #print("%s = %g"%(v.varName,v.x))
                ans_x[v.varName] = int(v.x)
            if "y" in v.varName:
                ans_y[v.varName] = int(v.x)
            if "b" in v.varName:
                ans_b[v.varName] = int(v.x)

        #print('Opt Value = ', model.ObjVal)
        print("x",ans_x)
        print("bin\t",sum(ans_y.values()))
        print("penalty\t",sum(ans_b.values())*r[0])
        print("Objective value\t",sum(ans_y.values())+(sum(ans_b.values()))*r[0]-sum(r))

        return model.ObjVal"""

In [6]:
#入力, 出力を書き出す階層を指定
path = os.path.abspath(".")
path

'/Users/yutaokamoto/Desktop/yutaokamoto_mac/Seminar/benchmark'

In [ ]:
for instance_scale in ["w120","w250","w500"]:
    for instance_number in range(10):
        for density in range(10):
            
            if (instance_number==0):################################
                
                n = 0
                V = []
                C = 150
                w = []
                E = []
                M = []
                group_split_cost = 1
                r = []
                bin_cost = 5
                ans_x = []
                ans_y = []
                
                file_input = path + "/" + "input" + "/" + str("i"+str(instance_scale[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + ".txt" #インスタンスの規模_インスタンスの番号_グラフの密度
                with open(file_input,mode="r") as f:
                    for line in f:
                        if "Number of items\t" in line:
                            n = int(line.split("\t")[-2])
                            V = list(range(n))
                        elif "Capacity of bins\t" in line:
                            C = int(line.split("\t")[-2])
                        elif "w\t" in line:
                            w.append(int(line.split("\t")[-1]))
                        elif "e\t" in line:
                            E.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").split(","))))
                        elif "g\t" in line:
                            M.append(tuple(map(int,line.split("\t")[-1].rstrip("\n").strip("(").strip(")").strip(",").split(","))))
                        elif "Costs of spliting groups\t" in line:
                            r = [int(i) for i in line.split("\t")[-2][1:-1].split(",")]
                        elif "Costs of using bins\t" in line:
                            bin_cost = int(line.split("\t")[-2])
                            
                #print(M[0])
                
                #コンフリクト
                conf={}
                n_conf = len(E)
                for i in range(n_conf):
                    temp = [0]*n
                    temp[E[i][0]-1] = 1
                    temp[E[i][1]-1] = 1
                    conf[f"conf{i}"] = temp

                file_output =  path + "/" + "output" + "/" + "Gurobi" + "/" + str("i"+str(instance_scale[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + "_" + str(tl) + "s"# + ".txt"
                model_name = str(file_output)

                objval = 0
                r = list(map(lambda x:x/bin_cost,r))
                bin_cost = bin_cost/bin_cost
                objval = solve(model_name,n,w,C,E,M,r,bin_cost,tl)
                
                """file_output =  path + "/" + "output" + "/" + "Gurobi" + "/" + str("i"+str(instance_scale[1:])) + "_"  + str("n"+str(instance_number)) + "_"  + str("d"+str(int(density*10))) + "_" + str(tl) + "s" + ".txt"
                with open(file_output,mode="w") as f:
                    f.write("Object value\t"+str(objval)+"\n\n")"""

Academic license - for non-commercial use only
Changed value of parameter TimeLimit to 1000.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 129840 rows, 117120 columns and 187320 nonzeros
Variable types: 0 continuous, 117120 integer (117120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 105.0000000
Presolve removed 107400 rows and 101880 columns
Presolve time: 1.64s
Presolved: 22440 rows, 15240 columns, 117360 nonzeros
Variable types: 0 continuous, 15240 integer (15240 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 1.55s

Solved with primal simplex

Root relaxation: objective 4.945333e+01, 14461 iterations, 4.49 seconds
Total elapsed time = 12.07s
Total elapsed time = 19.42s
Total elapsed time = 23.05s
Total elapsed time = 27.7

  1055   648   50.48871   47  296   54.20000   50.31069  7.18%  1327  698s
  1057   649   51.15665   73  309   54.20000   50.31069  7.18%  1325  713s
  1058   650   50.60933   23  479   54.20000   50.31069  7.18%  1323  718s
  1059   651   51.16023   80  471   54.20000   50.31069  7.18%  1322  720s
  1060   651   53.80000   54  592   54.20000   50.31069  7.18%  1321  727s
  1061   652   54.00000   90  578   54.20000   50.31069  7.18%  1320  730s
  1062   653   50.77020   57  564   54.20000   50.34478  7.11%  1318  739s
  1063   653   50.60154   55  529   54.20000   50.36214  7.08%  1317  740s
  1064   654   51.45781  113  584   54.20000   50.37659  7.05%  1316  750s
  1068   657   50.48871   42  468   54.20000   50.40000  7.01%  1311  755s
  1072   659   50.54000   32  249   54.20000   50.40000  7.01%  1306  763s
  1073   660   51.28576   79  349   54.20000   50.40000  7.01%  1305  765s
  1074   661   50.48617   18  267   54.20000   50.40000  7.01%  1304  774s
  1075   661   50.86876  

  1282   965   50.40000   30  306   55.20000   50.40000  8.70%   287  646s
  1287   970   50.40000   30  333   55.20000   50.40000  8.70%   296  652s
  1293   971   50.40000   31  317   55.20000   50.40000  8.70%   313  660s
  1300   974   50.93529   31  305   55.20000   50.40000  8.70%   333  668s
  1306   976   50.40000   32  309   55.20000   50.40000  8.70%   346  680s
  1318   979   50.40000   33  301   55.20000   50.40000  8.70%   375  692s
  1328   990   50.40000   34  309   55.20000   50.40000  8.70%   395  703s
  1340   998   50.80822   36  330   55.20000   50.40000  8.70%   422  714s
  1354  1006   50.63038   38  288   55.20000   50.40000  8.70%   457  725s
  1368  1011   50.81306   40  330   55.20000   50.40000  8.70%   485  740s
  1391  1026   50.82110   44  289   55.20000   50.40000  8.70%   522  754s
  1431  1057   50.84941   54  292   55.20000   50.40000  8.70%   581  771s
  1471  1072   50.73978   65  290   55.20000   50.40000  8.70%   653  788s
  1549  1126   51.14672  

    15    10   50.52000    4  299   55.20000   50.52000  8.48%  3063  361s
    23    12   50.52000    5  306   55.20000   50.52000  8.48%  2582  366s
    36    14   50.52000    7  291   55.20000   50.52000  8.48%  2073  371s
    55    24   50.52000   11  260   55.20000   50.52000  8.48%  1618  376s
    73    32 infeasible   13        55.20000   50.52000  8.48%  1446  380s
    91    40   50.69222   15  266   55.20000   50.52000  8.48%  1366  385s
   114    54   50.68090   19  254   55.20000   50.52000  8.48%  1317  394s
   128    62   50.68090   23  263   55.20000   50.52000  8.48%  1311  398s
   147    71 infeasible   28        55.20000   50.52000  8.48%  1294  402s
   167    81   50.68090   32  271   55.20000   50.52000  8.48%  1263  407s
   196    96 infeasible   36        55.20000   50.52000  8.48%  1189  411s
   213   101   51.16090   40  278   55.20000   50.52000  8.48%  1234  418s
   238   121   51.45666   44  267   55.20000   50.52000  8.48%  1252  424s
   258   130   51.16204  

   209    33 infeasible   28        56.20000   51.05333  9.16%  1631  257s
   254    36   51.05333   34  397   56.20000   51.05333  9.16%  1493  260s
   328    55 infeasible   43        56.20000   51.05333  9.16%  1406  266s
   372    74   51.80000   50  296   56.20000   51.05333  9.16%  1437  270s
   435   127   52.80000   55  367   56.20000   51.05333  9.16%  1592  278s
   474   175   51.80000   58  308   56.20000   51.05333  9.16%  1665  282s
   514   211   52.00000   63  218   56.20000   51.05333  9.16%  1761  287s
   577   276   52.20000   72  239   56.20000   51.05333  9.16%  1805  318s
   637   341   52.20000   77  224   56.20000   51.05333  9.16%  1819  324s
   839   528   52.21565   86  237   56.20000   51.05333  9.16%  1622  327s
  1055   742   52.70290  105  180   56.20000   51.05333  9.16%  1431  331s
H 1368  1041                      56.0000000   51.05333  8.83%  1182  339s
  1369  1039   53.52639   91  370   56.00000   51.05333  8.83%  1181  355s
H 1370   988             

...

Changed value of parameter TimeLimit to 1000.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 33939000 rows, 2761000 columns and 65876500 nonzeros
Variable types: 0 continuous, 2761000 integer (2761000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 494.2000000
Presolve removed 0 rows and 0 columns (presolve time = 17s) ...
Presolve removed 2500000 rows and 2505000 columns (presolve time = 21s) ...
Presolve removed 2500000 rows and 2505000 columns (presolve time = 27s) ...
Presolve removed 2500000 rows and 2505000 columns (presolve time = 92s) ...
Presolve removed 2500000 rows and 2505000 columns (presolve time = 123s) ...
Presolve removed 2500000 rows and 2505000 columns (presolve time = 128s) ...
Presolve removed 2500000 rows and 2505000 columns (presolve time = 130s) ...
Presolve removed 2500000 row

Presolve removed 28235980 rows and 2505000 columns (presolve time = 605s) ...
Presolve removed 28252686 rows and 2505000 columns (presolve time = 610s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 615s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 620s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 625s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 630s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 635s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 640s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 645s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 653s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 655s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve time = 660s) ...
Presolve removed 28262628 rows and 2505000 columns (presolve tim

Presolve removed 35857485 rows and 2505000 columns (presolve time = 420s) ...
Presolve removed 35897102 rows and 2505000 columns (presolve time = 425s) ...
Presolve removed 35936520 rows and 2505000 columns (presolve time = 430s) ...
Presolve removed 35988760 rows and 2505000 columns (presolve time = 435s) ...
Presolve removed 36028586 rows and 2505000 columns (presolve time = 440s) ...
Presolve removed 36073642 rows and 2505000 columns (presolve time = 445s) ...
Presolve removed 36111852 rows and 2505000 columns (presolve time = 450s) ...
Presolve removed 36149760 rows and 2505000 columns (presolve time = 455s) ...
Presolve removed 36195132 rows and 2505000 columns (presolve time = 460s) ...
Presolve removed 36230868 rows and 2505000 columns (presolve time = 465s) ...
Presolve removed 36288748 rows and 2505000 columns (presolve time = 470s) ...
Presolve removed 36329375 rows and 2505000 columns (presolve time = 475s) ...
Presolve removed 36390618 rows and 2505000 columns (presolve tim

Presolve removed 38134208 rows and 2505000 columns (presolve time = 950s) ...
Presolve removed 38134208 rows and 2505000 columns (presolve time = 955s) ...
Presolve removed 38134208 rows and 2505000 columns (presolve time = 960s) ...
Presolve removed 38134208 rows and 2505000 columns (presolve time = 965s) ...
Presolve removed 38134208 rows and 2505000 columns (presolve time = 970s) ...
Presolve removed 38134208 rows and 2505000 columns (presolve time = 975s) ...
Presolve removed 38134208 rows and 2505000 columns
Presolve time: 979.02s

Explored 0 nodes (0 simplex iterations) in 1005.82 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 516.6 

Time limit reached
Best objective 5.166000000000e+02, best bound -, gap -
Changed value of parameter TimeLimit to 1000.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 54548500 rows, 2761000 columns and 107095500 nonzeros
Variable types: 0 continuous, 2761000 integer (2761000 binary)
Coeffici

Presolve removed 49223830 rows and 2505000 columns (presolve time = 710s) ...
Presolve removed 49333528 rows and 2505000 columns (presolve time = 715s) ...
Presolve removed 49448393 rows and 2505000 columns (presolve time = 720s) ...
Presolve removed 49567341 rows and 2505000 columns (presolve time = 725s) ...
Presolve removed 49686838 rows and 2505000 columns (presolve time = 730s) ...
Presolve removed 49807116 rows and 2505000 columns (presolve time = 735s) ...
Presolve removed 49925680 rows and 2505000 columns (presolve time = 740s) ...
Presolve removed 50049920 rows and 2505000 columns (presolve time = 745s) ...
Presolve removed 50184355 rows and 2505000 columns (presolve time = 750s) ...
Presolve removed 50256438 rows and 2505000 columns (presolve time = 755s) ...
Presolve removed 50373996 rows and 2505000 columns (presolve time = 760s) ...
Presolve removed 50502256 rows and 2505000 columns (presolve time = 765s) ...
Presolve removed 50635560 rows and 2505000 columns (presolve tim

# 解の図示 

## ビンごとに辞書に登録 

In [21]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        try:
            node_dict[Bin] = node_dict[Bin] + [Item]
        except:
            node_dict[Bin] = [Item]
        
node_dict

{0: [0]}

In [16]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        node_dict.setdefault(Bin,[])
        node_dict[Bin] = node_dict[Bin] + [Item]
    
node_dict

{0: [0]}

In [17]:
node_dict = {}
for Item,Bin in ans_x:
    if ans_x[Item,Bin] == 1:
        Item = int(Item)
        Bin = int(Bin)
        node_dict[Bin] = node_dict.setdefault(Bin,[]) + [Item]
    
node_dict

{0: [0]}

## 描画

In [18]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
pos = nx.spring_layout(G, k=1)
plt.figure(figsize=(10,10))


for l in node_dict.values():
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    nx.draw_networkx_nodes(G, pos, nodelist=[i for i in l], node_color=node_color, alpha=0.6, node_size=4000)
    nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")
    
%matplotlib inline
plt.show()
#plt.savefig("figure.png")

In [19]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
plt.figure(figsize=(10,10))

for b in range(n):
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    for i in range(n):
        if ans_x[str(i),str(b)]==1:
            nx.draw_networkx_nodes(G, pos, nodelist=[i],node_color=node_color, alpha=0.6, node_size=4000)
            nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")

plt.show()
#plt.savefig("figure.png")

KeyError: ('10', '0')

findfont: Font family ['Yu Gothic'] not found. Falling back to DejaVu Sans.


In [ ]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import random
G = nx.Graph()

for v in range(n): # グラフにノードを追加
    G.add_node(v)
    
plt.figure(figsize=(10,10))

for b in map(lambda x:x[1],ans_x.keys()):
    node_color = random.sample(matplotlib.colors.cnames.keys(),1)
    for i in list(set(map(lambda x:x[0],ans_x.keys()))):
        if ans_x[str(i),str(b)]==1:
            nx.draw_networkx_nodes(G, pos, nodelist=[int(i)],node_color=node_color, alpha=0.6, node_size=4000)
            nx.draw_networkx_labels(G, pos, font_size=50, font_family="Yu Gothic", font_weight="bold")

plt.show()
#plt.savefig("figure.png")